# Physiological Signal Analysis

### Import packages

In [ ]:
import neurokit2 as nk
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline

### Specify data path and initialize variables

In [ ]:
sampling_rate = 250.0
data_path = r'data/'
signal_path = os.path.join(data_path, 'P01_S1_mathEasy_1674693325_07705.csv')


### Load the data and plot signal

In [ ]:
eda = []
resp = []
ppg1 = []
ppg2 =[]
arduino_ts = []
event_code = []
skip_first = True
with open(signal_path, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for ln in csvreader:
        if skip_first:
            print(ln)
            skip_first = False
        else:
            # print(ln)
            # break
            eda.append(float(ln[0]))
            resp.append(float(ln[1]))
            ppg1.append(float(ln[2]))
            ppg2.append(float(ln[3]))
            arduino_ts.append(float(ln[4]))
            if ln[5] != '':
                event_code.append(float(ln[5]))
            else:
                event_code.append(-1)

eda = np.array(eda)
resp = np.array(resp)
ppg1 = np.array(ppg1)
ppg2 = np.array(ppg2)
arduino_ts = np.array(arduino_ts)
arduino_ts = (arduino_ts - arduino_ts[0])/1000
event_code = np.array(event_code)
# plt.plot(np.arange(0, len(raw_signal)/sampling_rate, 1/sampling_rate), raw_signal)

In [ ]:
plt.plot(arduino_ts, ppg1, 'g')
plt.plot(arduino_ts[event_code != -1], ppg1[event_code != -1], 'ro', markersize=2)
plt.xlabel("Time (seconds)")
plt.ylabel("Signal Amplitude")


***
## Process PPG Signal
***

In [ ]:
signals, info = nk.ppg_process(ppg1, sampling_rate=sampling_rate)
plt.plot(np.arange(0, len(signals['PPG_Clean'])/sampling_rate, 1/sampling_rate), signals['PPG_Clean'])
plt.plot(info['PPG_Peaks']/sampling_rate, signals['PPG_Clean'][info['PPG_Peaks']], 'go')
plt.xlabel("Time (seconds)")
plt.ylabel("Normalized Signal Amplitude")

In [ ]:
hrv_indices = nk.hrv(info['PPG_Peaks'], sampling_rate=sampling_rate, show=True)
hrv_indices


***
## Process EDA Signal
***

In [ ]:
signals, info = nk.eda_process(eda, sampling_rate=sampling_rate)
# Extract clean EDA and SCR features
cleaned = signals["EDA_Clean"]
features = [info["SCR_Onsets"], info["SCR_Peaks"], info["SCR_Recovery"]]

In [ ]:
# Plot EDA signal
nk.eda_plot(signals)

***
## Process Respiratory Signal
***

In [ ]:
# Clean signal
cleaned = nk.rsp_clean(resp, sampling_rate=sampling_rate)

# Extract peaks
df, peaks_dict = nk.rsp_peaks(cleaned) 
info = nk.rsp_fixpeaks(peaks_dict)
formatted = nk.signal_formatpeaks(info, desired_length=len(cleaned),peak_indices=info["RSP_Peaks"])

In [ ]:
fixed_peaks = nk.events_plot(info['RSP_Peaks'], cleaned)

In [ ]:
# Extract rate
rsp_rate = nk.rsp_rate(cleaned, peaks_dict, sampling_rate=sampling_rate)

# Visualize
nk.signal_plot(rsp_rate, sampling_rate=sampling_rate)
plt.ylabel('Breaths Per Minute')

In [ ]:
rrv = nk.rsp_rrv(rsp_rate, info, sampling_rate=sampling_rate, show=True)
rrv